In [ ]:
!pip install --upgrade google-cloud-vision


In [ ]:
!pip install utils

In [ ]:
import os
import io
import numpy as np
import platform
from PIL import ImageFont, ImageDraw, Image
import utils
import matplotlib.pyplot as plt
import cv2
from google.cloud import vision
from google.cloud import vision_v1


In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'ultra-airway-412008-b349834297f5.json'

client_options = {'api_endpoint': 'eu-vision.googleapis.com'}
client = vision.ImageAnnotatorClient(client_options=client_options)

In [ ]:
from google.cloud import vision
import pandas as pd
df = pd.read_csv('data.csv')

## 썸네일 텍스트 컬럼 추가 코드
1. 이미지 파일로 추가
2. 썸네일 url로 추가


In [ ]:
#이미지 파일
from google.cloud import vision_v1

def ocr_with_google_cloud_vision(image_url):
    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations

    if texts:
        return texts[0].description
    else:
        return ''

for idx, col in 이미지 경로 리스트
    df.loc[idx, '썸네일 문자'] = ocr_with_google_cloud_vision(col)

In [ ]:
#url로 추가
from google.cloud import vision_v1

def ocr_with_google_cloud_vision(image_url):
    client = vision_v1.ImageAnnotatorClient()
    image = vision_v1.Image()
    image.source.image_uri = image_url

    response = client.text_detection(image=image)
    texts = response.text_annotations

    if texts:
        # 첫 번째 감지된 텍스트를 반환합니다.
        return texts[0].description
    else:
        return ''

# DataFrame을 반복하며 '썸네일 문자' 열을 업데이트합니다.
# for idx, col in enumerate(df['썸네일 주소']):
    # df.loc[idx, '썸네일 문자'] = ocr_with_google_cloud_vision(col)

## 유해성 수치 컬럼 추가 코드


In [ ]:
def detect_safe_search(image_url):
    """Detects unsafe features in the file."""
    from google.cloud import vision

    client = vision_v1.ImageAnnotatorClient()
    image = vision_v1.Image()
    image.source.image_uri = image_url



    response = client.safe_search_detection(image=image)
    safe = response.safe_search_annotation

    # Names of likelihood from google.cloud.vision.enums
    likelihood_name = (0,10,20,30,40,50)
    # print("Safe search:")

    # print(f"adult: {likelihood_name[safe.adult]}")
    # print(f"medical: {likelihood_name[safe.medical]}")
    # print(f"spoofed: {likelihood_name[safe.spoof]}")
    # print(f"violence: {likelihood_name[safe.violence]}")
    # print(f"racy: {likelihood_name[safe.racy]}")
    # print(f"adult: {likelihood_name[safe.adult]}")
    # print(f"medical: {likelihood_name[safe.medical]}")
    # print(f"spoofed: {likelihood_name[safe.spoof]}")
    # print(f"violence: {likelihood_name[safe.violence]}")
    # print(f"racy: {likelihood_name[safe.racy]}")
    score = likelihood_name[safe.adult]+likelihood_name[safe.medical]+likelihood_name[safe.spoof]+likelihood_name[safe.violence]+likelihood_name[safe.racy]


    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    return score
for idx, col in enumerate(df['썸네일 주소']):
    df.loc[idx, '유해성 수치'] = detect_safe_search(col)

In [ ]:
df

## RGB 값 컬럼 추가 코드

In [ ]:
def detect_properties(image_url):
    """Detects image properties in the file."""
    from google.cloud import vision
    client = vision_v1.ImageAnnotatorClient()
    image = vision_v1.Image()
    image.source.image_uri = image_url


    response = client.image_properties(image=image)
    props = response.image_properties_annotation
    # print("Properties:")
    fraction = []
    max = 0
    cnt = 0
    for color in props.dominant_colors.colors:
        fraction.append([color.pixel_fraction,color.color.red,color.color.green,color.color.blue])
    for idx,i in enumerate(fraction):
        if max < fraction[idx][0]:
            max = fraction[idx][0]
            cnt = idx
    # print(fraction[cnt])
    # print(props)
    # print(fraction)
    # print(fraction[cnt])

        # print(f"\tr: {color.color.red}")
        # print(f"\tg: {color.color.green}")
        # print(f"\tb: {color.color.blue}")
        # print(f"\ta: {color.color.alpha}")

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    return fraction[cnt]
for idx, col in enumerate(df['썸네일 주소']):
    df.loc[idx, 'R'] = detect_properties(col)[1]
    df.loc[idx, 'G'] = detect_properties(col)[2]
    df.loc[idx, 'B'] = detect_properties(col)[3]

df

## 썸네일 텍스트 상중하 위치

In [ ]:
img_path = []
with io.open(path, 'rb') as image_file:
    content = image_file.read()
image = vision.Image(content=content)
response = client.text_detection(image=image)
texts = response.text_annotations

def process_image_with_text(img_path, texts):
    img = cv2.imread(img_path)
    roi_img = img.copy()
    height = img.shape[0]
    height1 = height * (1 / 3)
    height2 = height * (2 / 3)
    cnt = {
        'up': 0, 'middle': 0, 'down': 0,'none':0
    }

    if not texts:  # 텍스트가 없는 경우
        cnt['none']+=1
    else:
        for text in texts[1:-1]:
            print('\n"{}"'.format(text.description))

            vertices = (['({},{})'.format(vertex.x, vertex.y)
                         for vertex in text.bounding_poly.vertices])

            ocr_text = text.description
            y1 = text.bounding_poly.vertices[0].y
            y2 = text.bounding_poly.vertices[2].y

            m_y = (abs(y2 + y1)) // 2
            print(m_y)
            if 0 <= m_y < height1:
                cnt['up'] += len(ocr_text)
            elif height1 <= m_y < height2:
                cnt['middle'] += len(ocr_text)
            elif height2 <= m_y:
                cnt['down'] += len(ocr_text)
        if response.error.message:
            raise Exception(
                '{}\nFor more info on error messages, check: '
                'https://cloud.google.com/apis/design/errors'.format(
                    response.error.message))
    return(max(cnt))

for idx, col in 이미지파일 경로:
    df.loc[idx, '텍스트 위치'] = process_image_with_text(col,texts)

## 사람 수 카운트 (얼굴 기준)

In [ ]:
def detect_faces_uri(image_path):


    client = vision.ImageAnnotatorClient()

    # 이미지 파일을 읽어옵니다.
    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    # 이미지 데이터를 Vision API로 전송합니다.
    image = vision.Image(content=content)

    response = client.face_detection(image=image)
    faces = response.face_annotations

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    return len(faces)
for idx, col in 이미지파일 경로:
    df.loc[idx, '썸네일 사람수'] = detect_faces_uri(image_path)
df[(df['썸네일 사람수']<3) & (df['썸네일 사람수']>=1)] = 1
df[df['썸네일 사람수']>=3] = 2